**Importing The Required Libraries**

In [ ]:
import pandas as pd
!pip install catboost
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import sys
!{sys.executable} -m pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 8.2 MB/s eta 0:00:00


**Reading the Data**

In [4]:
df = pd.read_csv('/content/Final_Data.csv')
df.head()

,season,match_id,match_name,home_team,away_team,venue,batting_innings,bowling_innings,fullName,Total_FP,Dream_Team,Captain,Vice_Captain,Against Team,Role
0,2023,1370353,GT v CSK,GT,CSK,"Narendra Modi Stadium, Motera, Ahmedabad",1,2,Sai Sudharsan,134,1,1,0,CSK,Batsman
1,2023,1370353,GT v CSK,GT,CSK,"Narendra Modi Stadium, Motera, Ahmedabad",1,2,Mohit Sharma,95,1,0,1,CSK,Bowler
2,2023,1370353,GT v CSK,GT,CSK,"Narendra Modi Stadium, Motera, Ahmedabad",1,2,Shubman Gill,60,1,0,0,CSK,Batsman
3,2023,1370353,GT v CSK,GT,CSK,"Narendra Modi Stadium, Motera, Ahmedabad",1,2,Hardik Pandya,35,0,0,0,CSK,All-Rounder
4,2023,1370353,GT v CSK,GT,CSK,"Narendra Modi Stadium, Motera, Ahmedabad",1,2,David Miller,12,0,0,0,CSK,Batsman


In [5]:
df.shape

(1854, 15)

**Drop the Features which are not required**

In [6]:
df1=df.drop(['season','match_id','match_name','home_team','away_team','bowling_innings','Captain','Vice_Captain','Role'],axis=1)

In [7]:
df1.head()

,venue,batting_innings,fullName,Total_FP,Dream_Team,Against Team
0,"Narendra Modi Stadium, Motera, Ahmedabad",1,Sai Sudharsan,134,1,CSK
1,"Narendra Modi Stadium, Motera, Ahmedabad",1,Mohit Sharma,95,1,CSK
2,"Narendra Modi Stadium, Motera, Ahmedabad",1,Shubman Gill,60,1,CSK
3,"Narendra Modi Stadium, Motera, Ahmedabad",1,Hardik Pandya,35,0,CSK
4,"Narendra Modi Stadium, Motera, Ahmedabad",1,David Miller,12,0,CSK


**Train-Test Splitting**

In [8]:
X = df1[['venue', 'fullName', 'Total_FP', 'batting_innings', 'Against Team']]
y = df1['Dream_Team']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (1483, 5)
X_test shape: (371, 5)
y_train shape: (1483,)
y_test shape: (371,)


**Model Training**

In [9]:
categorical_features_indices = ['venue', 'fullName', 'Against Team']

# Initialize CatBoostClassifier
cat_model = CatBoostClassifier(random_state=42, verbose=1, iterations=100)

# Train the model
cat_model.fit(X_train, y_train, cat_features=categorical_features_indices)


Learning rate set to 0.100696
0:	learn: 0.6262178	total: 49.3ms	remaining: 4.88s
1:	learn: 0.5689569	total: 50.5ms	remaining: 2.47s
2:	learn: 0.5230940	total: 51.1ms	remaining: 1.65s
3:	learn: 0.4825199	total: 52ms	remaining: 1.25s
4:	learn: 0.4487206	total: 52.7ms	remaining: 1s
5:	learn: 0.4194683	total: 54.1ms	remaining: 848ms
6:	learn: 0.3933522	total: 55.2ms	remaining: 733ms
7:	learn: 0.3707713	total: 56ms	remaining: 644ms
8:	learn: 0.3526705	total: 57.3ms	remaining: 579ms
9:	learn: 0.3350992	total: 58.6ms	remaining: 527ms
10:	learn: 0.3210384	total: 59.1ms	remaining: 479ms
11:	learn: 0.3090399	total: 59.8ms	remaining: 438ms
12:	learn: 0.2969531	total: 60.9ms	remaining: 408ms
13:	learn: 0.2850996	total: 62.2ms	remaining: 382ms
14:	learn: 0.2767321	total: 62.8ms	remaining: 356ms
15:	learn: 0.2666353	total: 63.4ms	remaining: 333ms
16:	learn: 0.2600371	total: 64.6ms	remaining: 315ms
17:	learn: 0.2527920	total: 70.1ms	remaining: 319ms
18:	learn: 0.2470740	total: 70.8ms	remaining: 302ms

**Model Evaluation**

In [10]:
# Make predictions on the test set
y_pred = cat_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Accuracy: 0.9380
Precision: 0.9648
Recall: 0.9231
F1-Score: 0.9435


**Tuning Model Hyperparameters using Optuna**

In [11]:
def objective(trial):

    params = {
        'iterations': trial.suggest_int('iterations', 100, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15, log=True),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 100.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.05, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.05, 1.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        'random_seed': 42 # Ensure reproducibility for each trial
    }


    model = CatBoostClassifier(**params, verbose=1)


    model.fit(X_train, y_train, cat_features=categorical_features_indices)


    y_pred_trial = model.predict(X_test)


    f1 = f1_score(y_test, y_pred_trial)


    return f1

print("Objective function for Optuna defined.")

Objective function for Optuna defined.


In [12]:
import optuna

study = optuna.create_study(direction='maximize')

study.optimize(objective, n_trials=10)


print("\nNumber of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print(f"  Value (F1-score): {trial.value:.4f}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2026-02-04 16:11:47,973] A new study created in memory with name: no-name-2f67b8c4-2cfc-4f51-8cac-b56f1fecf7e3


0:	learn: 0.6450443	total: 4.68ms	remaining: 1.38s
1:	learn: 0.5849263	total: 6.68ms	remaining: 978ms
2:	learn: 0.5371100	total: 10ms	remaining: 978ms
3:	learn: 0.4938403	total: 12ms	remaining: 875ms
4:	learn: 0.4558046	total: 13.9ms	remaining: 803ms
5:	learn: 0.4554206	total: 15.5ms	remaining: 747ms
6:	learn: 0.4179809	total: 18.4ms	remaining: 757ms
7:	learn: 0.4174887	total: 21.2ms	remaining: 761ms
8:	learn: 0.3931334	total: 23.5ms	remaining: 746ms
9:	learn: 0.3929454	total: 25.3ms	remaining: 720ms
10:	learn: 0.3671334	total: 28.4ms	remaining: 733ms
11:	learn: 0.3670166	total: 30.1ms	remaining: 709ms
12:	learn: 0.3538107	total: 32.9ms	remaining: 714ms
13:	learn: 0.3369259	total: 35.8ms	remaining: 718ms
14:	learn: 0.3244422	total: 38.9ms	remaining: 726ms
15:	learn: 0.3026217	total: 41.1ms	remaining: 717ms
16:	learn: 0.2892741	total: 43.7ms	remaining: 715ms
17:	learn: 0.2813786	total: 46.5ms	remaining: 716ms
18:	learn: 0.2811972	total: 49.1ms	remaining: 714ms
19:	learn: 0.2809779	total

[I 2026-02-04 16:11:48,807] Trial 0 finished with value: 0.9455445544554455 and parameters: {'iterations': 295, 'learning_rate': 0.06831154603616067, 'depth': 5, 'l2_leaf_reg': 78.24737113710009, 'subsample': 0.4912896525605437, 'colsample_bylevel': 0.44503687346261905, 'min_data_in_leaf': 20}. Best is trial 0 with value: 0.9455445544554455.


229:	learn: 0.1599710	total: 588ms	remaining: 166ms
230:	learn: 0.1598654	total: 593ms	remaining: 164ms
231:	learn: 0.1598654	total: 596ms	remaining: 162ms
232:	learn: 0.1597294	total: 598ms	remaining: 159ms
233:	learn: 0.1596693	total: 600ms	remaining: 157ms
234:	learn: 0.1594908	total: 603ms	remaining: 154ms
235:	learn: 0.1594377	total: 606ms	remaining: 151ms
236:	learn: 0.1594377	total: 607ms	remaining: 149ms
237:	learn: 0.1593050	total: 610ms	remaining: 146ms
238:	learn: 0.1593050	total: 611ms	remaining: 143ms
239:	learn: 0.1592914	total: 614ms	remaining: 141ms
240:	learn: 0.1592415	total: 616ms	remaining: 138ms
241:	learn: 0.1591504	total: 618ms	remaining: 135ms
242:	learn: 0.1590914	total: 621ms	remaining: 133ms
243:	learn: 0.1590541	total: 623ms	remaining: 130ms
244:	learn: 0.1588525	total: 625ms	remaining: 128ms
245:	learn: 0.1587854	total: 628ms	remaining: 125ms
246:	learn: 0.1586923	total: 631ms	remaining: 123ms
247:	learn: 0.1586746	total: 633ms	remaining: 120ms
248:	learn: 

[I 2026-02-04 16:11:49,443] Trial 1 finished with value: 0.9507389162561576 and parameters: {'iterations': 223, 'learning_rate': 0.029394084614450675, 'depth': 5, 'l2_leaf_reg': 20.536179092444815, 'subsample': 0.36037687364680004, 'colsample_bylevel': 0.4128503814450865, 'min_data_in_leaf': 100}. Best is trial 1 with value: 0.9507389162561576.


...

55:	learn: 0.3828223	total: 70ms	remaining: 136ms
56:	learn: 0.3798681	total: 75.8ms	remaining: 144ms
57:	learn: 0.3797906	total: 77.2ms	remaining: 142ms
58:	learn: 0.3766795	total: 78.5ms	remaining: 141ms
59:	learn: 0.3736276	total: 80.1ms	remaining: 140ms
60:	learn: 0.3705231	total: 81ms	remaining: 138ms
61:	learn: 0.3677111	total: 82.6ms	remaining: 137ms
62:	learn: 0.3650520	total: 83.7ms	remaining: 136ms
63:	learn: 0.3624443	total: 84.3ms	remaining: 133ms
64:	learn: 0.3600341	total: 85.4ms	remaining: 131ms
65:	learn: 0.3575641	total: 86.2ms	remaining: 129ms
66:	learn: 0.3549074	total: 87.4ms	remaining: 128ms
67:	learn: 0.3522591	total: 88.4ms	remaining: 126ms
68:	learn: 0.3495125	total: 90ms	remaining: 125ms
69:	learn: 0.3471011	total: 91.5ms	remaining: 124ms
70:	learn: 0.3445333	total: 93ms	remaining: 123ms
71:	learn: 0.3421209	total: 94.6ms	remaining: 122ms
72:	learn: 0.3399193	total: 96.3ms	remaining: 121ms
73:	learn: 0.3379170	total: 98ms	remaining: 120ms
74:	learn: 0.3357808	t

[I 2026-02-04 16:11:56,861] Trial 8 finished with value: 0.9408866995073891 and parameters: {'iterations': 181, 'learning_rate': 0.11046001721415485, 'depth': 5, 'l2_leaf_reg': 3.1297240564639415, 'subsample': 0.7014522704184333, 'colsample_bylevel': 0.5097017862457293, 'min_data_in_leaf': 46}. Best is trial 5 with value: 0.9533169533169533.


18:	learn: 0.2564723	total: 14.4ms	remaining: 122ms
19:	learn: 0.2483930	total: 18.1ms	remaining: 145ms
20:	learn: 0.2437916	total: 21.1ms	remaining: 161ms
21:	learn: 0.2383587	total: 22.1ms	remaining: 160ms
22:	learn: 0.2338643	total: 24ms	remaining: 165ms
23:	learn: 0.2300953	total: 24.8ms	remaining: 162ms
24:	learn: 0.2265767	total: 25.4ms	remaining: 159ms
25:	learn: 0.2235638	total: 26ms	remaining: 155ms
26:	learn: 0.2205612	total: 26.8ms	remaining: 153ms
27:	learn: 0.2160671	total: 27.4ms	remaining: 150ms
28:	learn: 0.2132152	total: 28.4ms	remaining: 149ms
29:	learn: 0.2107433	total: 29.3ms	remaining: 147ms
30:	learn: 0.2086474	total: 30.1ms	remaining: 145ms
31:	learn: 0.2069047	total: 30.8ms	remaining: 144ms
32:	learn: 0.2048807	total: 31.3ms	remaining: 141ms
33:	learn: 0.2030704	total: 31.9ms	remaining: 138ms
34:	learn: 0.2013591	total: 32.4ms	remaining: 135ms
35:	learn: 0.1991695	total: 33.3ms	remaining: 134ms
36:	learn: 0.1980100	total: 33.8ms	remaining: 132ms
37:	learn: 0.196

[I 2026-02-04 16:11:57,016] Trial 9 finished with value: 0.9512195121951219 and parameters: {'iterations': 167, 'learning_rate': 0.022482855536731936, 'depth': 6, 'l2_leaf_reg': 68.57791990848247, 'subsample': 0.6552918514977054, 'colsample_bylevel': 0.3598551072236038, 'min_data_in_leaf': 5}. Best is trial 5 with value: 0.9533169533169533.


2:	learn: 0.6731261	total: 2.31ms	remaining: 126ms
3:	learn: 0.6621436	total: 2.73ms	remaining: 111ms
4:	learn: 0.6527633	total: 3.58ms	remaining: 116ms
5:	learn: 0.6447464	total: 4.46ms	remaining: 120ms
6:	learn: 0.6444672	total: 4.87ms	remaining: 111ms
7:	learn: 0.6441560	total: 5.22ms	remaining: 104ms
8:	learn: 0.6439205	total: 5.84ms	remaining: 103ms
9:	learn: 0.6436295	total: 6.27ms	remaining: 98.4ms
10:	learn: 0.6335757	total: 6.92ms	remaining: 98.2ms
11:	learn: 0.6333055	total: 7.48ms	remaining: 96.6ms
12:	learn: 0.6297502	total: 8.5ms	remaining: 101ms
13:	learn: 0.6207498	total: 9.33ms	remaining: 102ms
14:	learn: 0.6204800	total: 9.89ms	remaining: 100ms
15:	learn: 0.6140081	total: 10.8ms	remaining: 102ms
16:	learn: 0.6138555	total: 11.6ms	remaining: 103ms
17:	learn: 0.6136266	total: 12ms	remaining: 99.5ms
18:	learn: 0.6134061	total: 12.6ms	remaining: 98.1ms
19:	learn: 0.6131935	total: 13ms	remaining: 95.3ms
20:	learn: 0.6129576	total: 13.5ms	remaining: 93.6ms
21:	learn: 0.60648

**Re-training the model on best hyperparameters**

In [13]:
best_params = study.best_params

optimal_cat_model = CatBoostClassifier(**best_params, random_seed=42, verbose=1)

optimal_cat_model.fit(X_train, y_train, cat_features=categorical_features_indices)

y_pred_optimal = optimal_cat_model.predict(X_test)

accuracy_optimal = accuracy_score(y_test, y_pred_optimal)
precision_optimal = precision_score(y_test, y_pred_optimal)
recall_optimal = recall_score(y_test, y_pred_optimal)
f1_optimal = f1_score(y_test, y_pred_optimal)

print("Optimal CatBoost model trained and evaluated.")
print(f"Accuracy (Optimal): {accuracy_optimal:.4f}")
print(f"Precision (Optimal): {precision_optimal:.4f}")
print(f"Recall (Optimal): {recall_optimal:.4f}")
print(f"F1-Score (Optimal): {f1_optimal:.4f}")

0:	learn: 0.6781924	total: 3.8ms	remaining: 1.13s
1:	learn: 0.6637146	total: 7.05ms	remaining: 1.04s
2:	learn: 0.6474940	total: 9.64ms	remaining: 948ms
3:	learn: 0.6321354	total: 12.5ms	remaining: 920ms
4:	learn: 0.6197746	total: 15.2ms	remaining: 892ms
5:	learn: 0.6053228	total: 17.8ms	remaining: 868ms
6:	learn: 0.5934171	total: 20.3ms	remaining: 844ms
7:	learn: 0.5811971	total: 23.1ms	remaining: 839ms
8:	learn: 0.5699122	total: 25.3ms	remaining: 811ms
9:	learn: 0.5586619	total: 28.3ms	remaining: 816ms
10:	learn: 0.5482538	total: 30.9ms	remaining: 807ms
11:	learn: 0.5374311	total: 33.8ms	remaining: 805ms
12:	learn: 0.5253404	total: 36.5ms	remaining: 800ms
13:	learn: 0.5137407	total: 39.2ms	remaining: 794ms
14:	learn: 0.5044050	total: 41.8ms	remaining: 789ms
15:	learn: 0.4940338	total: 44.1ms	remaining: 776ms
16:	learn: 0.4871895	total: 47ms	remaining: 777ms
17:	learn: 0.4789071	total: 49.2ms	remaining: 766ms
18:	learn: 0.4716303	total: 52ms	remaining: 764ms
19:	learn: 0.4640838	total: